In [1]:
from src.hardware import DMD, ALP4
import numpy as np
import ctypes as ct

In [2]:
dmd=DMD()

Loading library: C:\Program Files\ALP-4.3\ALP-4.3 API/x64/alp4395.dll
DMD found, resolution = 1024 x 768.


In [3]:
img = np.zeros((1, *dmd.shape), dtype=np.uint8)

In [4]:
dmd.frames

array([], shape=(0, 0, 0), dtype=bool)

In [16]:
dmd._frames = img

dmd._alp4.Halt()
dmd._alp4.FreeSeq()
dmd._alp4.SeqAlloc(nbImg=img.shape[0])
dmd._alp4.SeqPut(img)

dmd._alp4.SeqControl(ALP4.ALP_FLUT_MODE, ALP4.ALP_FLUT_9BIT)
dmd._alp4.SeqControl(ALP4.ALP_FLUT_ENTRIES9, img.shape[0]) # may need to set ALP_FLUT_ENTRIES9 to 2*len(sequence) according to C++ sample, although this would correspond to 18 bits
dmd._alp4.SeqControl(ALP4.ALP_BIN_MODE, ALP4.ALP_BIN_UNINTERRUPTED)
dmd._alp4.SetTiming()

In [15]:
dmd._alp4.SeqInquire(ALP4.ALP_FLUT_OFFSET9)

0

In [35]:
frame_index = 0

flut = ALP4.tFlutWrite(
    nOffset=ct.c_long(0),
    nSize=ct.c_long(1),
    FrameNumbers=(ct.c_ulong * 4096)(frame_index),
)

dmd._alp4.ProjControlEx(ALP4.ALP_FLUT_WRITE_9BIT, ct.byref(flut))

ALPError: One of the parameters is invalid.

In [38]:
flut.nSize + flut.nOffset <= dmd._alp4.ProjInquire(ALP4.ALP_FLUT_MAX_ENTRIES9)

True

In [39]:
dmd.free()